In [0]:
SET use_cached_result = false;
WITH customer_total_return AS 
(
    SELECT  cr_returning_customer_sk AS ctr_customer_sk
            ,ca_state AS ctr_state
            ,SUM(cr_return_amt_inc_tax) AS ctr_total_return
    FROM workspace.tpcds_sf1000_iceberg.catalog_returns
    ,       workspace.tpcds_sf1000_iceberg.date_dim
    ,       workspace.tpcds_sf1000_iceberg.customer_address
    WHERE   cr_returned_date_sk = d_date_sk
    AND     d_year = 2002
    AND     cr_returning_addr_sk = ca_address_sk
    GROUP BY cr_returning_customer_sk
              ,ca_state
)
SELECT  c_customer_id
        ,c_salutation
        ,c_first_name
        ,c_last_name
        ,ca_street_number
        ,ca_street_name
        ,ca_street_type
        ,ca_suite_number
        ,ca_city
        ,ca_county
        ,ca_state
        ,ca_zip
        ,ca_country
        ,ca_gmt_offset
        ,ca_location_type
        ,ctr_total_return
FROM customer_total_return ctr1
,       workspace.tpcds_sf1000_iceberg.customer_address
,       workspace.tpcds_sf1000_iceberg.customer
WHERE   ctr1.ctr_total_return > (
            SELECT  AVG(ctr_total_return) * 1.2
            FROM customer_total_return ctr2
            WHERE   ctr1.ctr_state = ctr2.ctr_state
        ) 
AND     ca_address_sk = c_current_addr_sk
AND     ca_state = 'MI'
AND     ctr1.ctr_customer_sk = c_customer_sk
ORDER BY c_customer_id,c_salutation,c_first_name,c_last_name,ca_street_number,ca_street_name,ca_street_type,ca_suite_number,ca_city,ca_county,ca_state,ca_zip,ca_country,ca_gmt_offset,ca_location_type,ctr_total_return
LIMIT   100
;